In [11]:
import json
import boto3
import pandas as pd
import ast

from utils.analysis_utils import classify_feedback_extractions
from utils.database import get_field_value
from pymongo import MongoClient
import certifi
import importlib

In [12]:
# TODO: Remove

import importlib
import utils.analysis_utils
import utils.database

# Reload the modules
importlib.reload(utils.analysis_utils)
importlib.reload(utils.database)

# Now you can import your functions
from utils.analysis_utils import classify_feedback_extractions
from utils.database import get_field_value


In [13]:
_secrets_manager_client = boto3.client("secretsmanager", region_name="eu-west-3")

_secrets = json.loads(
    _secrets_manager_client.get_secret_value(
        SecretId=f"Prod/alloreview"
    )["SecretString"]
)
MONGO_CONNECTION_STRING = (
    "mongodb+srv://alloreview:{}@feedbacksdev.cuwx1.mongodb.net".format(
        _secrets["mongodb"]["password"]
    )
)
mongo_client = MongoClient(MONGO_CONNECTION_STRING,tlsCAFile=certifi.where())

collection = mongo_client['feedbacks_db']['feedbacks_Prod']

# Generating Elementary Subjects

After creating all the extractions, our objective is to generate elementary subjects.

We use the following method:

1. For each extraction, we attempt to classify it among the existing elementary subjects.
2. If no subject matches, we generate a new one.
3. As we progress, we also verify that we are not generating duplicates.

## Process

We classify and generate our elementary subjects progressively:

1. Classify each extraction
2. Generate new subjects as needed
3. Check for duplicates

By the end of this process, after classifying everything, we will have generated all our subjects.

## Database Storage

The elementary subjects are pushed into the `elementary_subjects_dev` database.

In [20]:
BRAND = 'ditp_analysis'

BRAND_DESCR = '''
Feedbacks are from French public services.
'''

In [5]:
# getting feedbacks from mongo

from_mongo = pd.DataFrame(list(collection.aggregate([
    {
        '$match': {
            'brand': BRAND,
        },
    },
])))

from_mongo.shape

KeyboardInterrupt: 

In [14]:
def format_ligne(ligne):
    # Fonction interne pour gérer les valeurs manquantes
    def extraire_champ(champ, allow_empty=False):
        return champ if pd.notnull(champ) and (allow_empty or champ != 'N/A') else None

    # Champs obligatoires et facultatifs
    champs = [
        ("Intitulé Structure 1", ligne.get("intitule_structure_1"), False),
        ("Intitulé Structure 2", ligne.get("intitule_structure_2"), True),
        ("Tags Métiers", ligne.get("tags_metiers"), True),
        ("Pays de la demande", ligne.get("pays"), False)
    ]
    
    # Initialisation des lignes avec une phrase fixe
    lignes = ["Feedbacks are from French public services."]
    
    # Génération des lignes dynamiques si les champs sont présents
    for label, champ, allow_empty in champs:
        valeur = extraire_champ(champ, allow_empty)
        if valeur:
            lignes.append(f"{label}: {valeur}")
    
    # Retour du résultat formaté
    return "\n".join(lignes)

In [32]:
# Keeping only feedbacks with extractions and splitted_analysis_v2

# subdf = from_mongo[from_mongo['extractions'].notna()]
# subdf = subdf[subdf['splitted_analysis_v2'].notna()]
df = pd.read_csv('ditp_test_2.csv')
subdf = df
subdf['brand_context'] = subdf.apply(format_ligne, axis=1)
print(subdf.shape)

row = subdf.sample().iloc[0]

print("Example of row:", row.verbatims)


(43, 85)
Example of row: Passeport et social
Passeport délai pour rendez vous avec erreur de communications du consulat.
Dossier pour la CFE envoyé le 15 Septembre, toujours pas de réponse de réception ou corrections, seulement des e-mails automatique "ne répondez pas a cet e-mail"
PLUS de visas pour la famille du Français(e) directement au consulat, très coûteux et long en passant par une agence privée.
Consulats de pire en pire et de moins en moins de services, une honte française. Bonne journee si vraiment vous nous lisez....


The function `classify_feedback_extractions` is used to classify each extraction.

In [33]:
print(row["_id"])
print(row["extractions"])
print(row["brand_context"])

print(type(row["_id"]))
print(type(row["extractions"]))
print(type(row["brand_context"]))

ditp_analysis/385794
[{'sentiment': 'NEGATIVE', 'extraction': 'Délai pour rendez-vous avec erreur de communication du consulat', 'text': 'Passeport délai pour rendez vous avec erreur de communications du consulat.'}, {'sentiment': 'NEGATIVE', 'extraction': 'Pas de réponse de réception ou corrections pour le dossier CFE', 'text': 'Dossier pour la CFE envoyé le 15 Septembre,'}, {'sentiment': 'NEGATIVE', 'extraction': 'Pas de réponse de réception ou corrections pour le dossier CFE', 'text': ' toujours pas de réponse de réception ou corrections,'}, {'sentiment': 'NEGATIVE', 'extraction': 'E-mails automatiques sans réponse', 'text': ' seulement des e-mails automatique "ne répondez pas a cet e-mail"\r\n'}, {'sentiment': 'NEGATIVE', 'extraction': 'Coût et durée élevés pour obtenir des visas via une agence privée', 'text': 'PLUS de visas pour la famille du Français(e) directement au consulat,'}, {'sentiment': 'NEGATIVE', 'extraction': 'Coût et durée élevés pour obtenir des visas via une agence

In [34]:
res = classify_feedback_extractions(
    feedback_id=row['_id'],
    extractions=ast.literal_eval(row["extractions"]),
    model="gpt-4o-mini",
    brand_name=BRAND,
    brand_context=row["brand_context"],
    language="french",
    extractions_column="extractions",
    should_update_mongo=False
)

res

{'id': 'ditp_analysis/385794',
 'extractions': [{'sentiment': 'NEGATIVE',
   'extraction': 'Délai pour rendez-vous avec erreur de communication du consulat',
   'text': 'Passeport délai pour rendez vous avec erreur de communications du consulat.',
   'elementary_subjects': ['Service : Délai de Rendez-vous : Erreur de Communication'],
   'topics': {}},
  {'sentiment': 'NEGATIVE',
   'extraction': 'Pas de réponse de réception ou corrections pour le dossier CFE',
   'text': 'Dossier pour la CFE envoyé le 15 Septembre,',
   'elementary_subjects': ['Service : Communication : Absence de Réponse sur Dossier CFE'],
   'topics': {}},
  {'sentiment': 'NEGATIVE',
   'extraction': 'Pas de réponse de réception ou corrections pour le dossier CFE',
   'text': ' toujours pas de réponse de réception ou corrections,',
   'elementary_subjects': ['Service : Communication : Absence de Réponse sur Dossier CFE'],
   'topics': {}},
  {'sentiment': 'NEGATIVE',
   'extraction': 'E-mails automatiques sans répons

In [8]:
# this function allows to parallelize the analysis process and to save the results on the mongo database
from utils.analysis_utils import run_analysis_full_parallel

In [11]:

importlib.reload(utils.analysis_utils)

# Now you can import your functions
from utils.analysis_utils import run_analysis_full_parallel

In [9]:
to_analyse = subdf.sample(10) # test on a subset
print('To extract:', to_analyse.shape[0])

extractions_with_ids = to_analyse[['extractions', '_id']].to_dict(orient='records')

To extract: 10


In [12]:
analysis = run_analysis_full_parallel(
    extractions_with_ids,
    BRAND,
    brand_descr=BRAND_DESCR,
    language='french',
    model="gpt-4o-mini",
    save_to_mongo=False
)

analysis

t


Processing chunks: 100%|██████████| 1/1 [00:00<00:00, 10305.42it/s]


[[], [], [], [], [], [], [], [], [], []]

In [ ]:
analysis

[{'id': 'columbuscafe_test/5b6e404652649c8ae22e',
  'extractions': [{'sentiment': 'POSITIVE',
    'extraction': 'Appréciation générale',
    'text': '2👍',
    'elementary_subjects': ['Satisfaction générale : Volonté de revenir'],
    'topics': []}]},
 {'id': 'columbuscafe_test/72f750f6fe9766ce0e77',
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': 'Erreur dans la commande',
    'text': 'Encore une fois une erreur dans la commande,',
    'elementary_subjects': ['Service : Problème de traitement des commandes : Commande non traitée'],
    'topics': []},
   {'sentiment': 'NEGATIVE',
    'extraction': 'Problèmes récurrents avec les commandes sur Deliveroo',
    'text': ' comme à chaque commande réalisée sur Deliveroo.',
    'elementary_subjects': ['Service : Problème de traitement des commandes : Problèmes avec les commandes sur Deliveroo'],
    'topics': []},
   {'sentiment': 'POSITIVE',
    'extraction': 'Service généralement bon en boutique',
    'text': ' Cliente de la bou

## Check elementary subjects that we generated

In [ ]:
from utils.database import get_elementary_subjects

In [ ]:
positive_elementary_subjects = get_elementary_subjects(BRAND, 'positive')

len(positive_elementary_subjects)

21

In [ ]:
[x['elementary_subject'] for x in positive_elementary_subjects[:5]]

['Ambiance : Ambiance agréable',
 'Service : Accueil chaleureux',
 'Service : Problemlösung : Schnelle Problemlösung bei Fehlern',
 'Produits : Qualité des produits : Qualité de la nourriture',
 'Produits : Qualité des produits : Qualité du café']

In [ ]:
negative_elementary_subjects = get_elementary_subjects(BRAND, 'negative')

print(len(negative_elementary_subjects))

[x['elementary_subject'] for x in negative_elementary_subjects[:5]]

15


['Service : Comportement du personnel : Serveur impoli ou nerveux',
 'Magasin : Aménagement : Emplacement du mobilier inapproprié',
 "Accessibilité : Difficulté d'accès pour les fauteuils roulants",
 'Produits : Problème de qualité : Chocolat non consommable',
 'Prix : Tarifs élevés']

In [13]:
row = subdf.sample().iloc[0]
extractions = get_field_value(
    feedback_id=row['_id'],
    field_name='extractions',
)
res = classify_one_feedback(
    feedback_id=row['_id'],
    extractions=extractions,
    model="gpt-4o-mini",
    brand=BRAND,
    brand_descr=BRAND_DESCR,
    language="french",
    extractions_column="extractions",
    update_mongo=False
)

# res

Sentence : J’ai fait ma demande d’Aspa en juillet 2021.
Extraction : Demande d'Aspa en attente depuis juillet 2021
Elementary Subjects : ["Service : Délais d'attente pour les demandes : Demande d'Aspa en attente"]
Sentence : Mon dossier est complet et il est toujours en attente depuis !
Extraction : Demande d'Aspa en attente depuis juillet 2021
Elementary Subjects : ["Service : Délais de traitement des demandes : Demande d'Aspa en attente"]
Sentence : je suis passée par la CARSAT DE L’ORNE ou j’ai trouvé une super écoute !
Extraction : Bonne écoute de la CARSAT DE L’ORNE
Elementary Subjects : ["Service : Qualité de l'écoute des agents"]
Sentence : J’ai reçu un mail sur mon espace client me disant que mon dossier serait traité sous 7 jours ouvrés,
Extraction : Dossier non traité dans les délais annoncés
Elementary Subjects : ['Service : Délai de traitement : Dossier non traité dans les délais annoncés']
Sentence : et bien non !
Extraction : Dossier non traité dans les délais annoncés
El

In [ ]:
========================
========================
Sentence : Démarche prime à la conversion interminable.
Extraction : Démarche prime à la conversion interminable
Elementary Subjects : ['Services publics : Démarches administratives : Démarche prime à la conversion trop longue']
===========
Sentence : Des documents fournis sont encore illisibles soi-disant...
Extraction : Documents fournis illisibles
Elementary Subjects : ['Documents : Problème de lisibilité']
===========
Sentence : mais nous avons des nouvelles de notre dossier tous les deux mois.
Extraction : Mises à jour du dossier tous les deux mois
Elementary Subjects : ['Service : Fréquence des mises à jour de dossier']
